## Load Packages

In [ ]:
suppressWarnings({
  # Code block with multiple lines
  library(DT)
  library(nlme)
  library(gridExtra)
  library(data.table)
  library(plyr)
  library(dplyr)
  library(formattable)
  library(tidyr)
  library(MASS) 
  library(shinyjs)
  library(compiler)
  library(future.apply)
  library(PowerAnalysisIL)
  library(tictoc)
  library(viridis)
})



## Power Estimation

$$\pi = P(Rejecting H_0|H_0 is false)$$

Procedure:
1. Sepecify the Model
2. Define Parameters
3. Simulate Data
4. Test Hypothesis
5. Repeat Step2 and 3 Many Times
6. Calculate the percentage of which the null hypothesis being rejected 

## The Model

**Model 3:**
$$y_{ij}= \begin{cases}
            \beta_{w0i} + \beta_{w1i} \times |O|_{ij} + \beta_{w2i} \times PE_{ij} + \beta_{w3i} \times PE_{ij} \times |O|_{ij} + \beta_{\phi i} \times y_{i(j-1)} \mbox{ when } O_{ij} > 0 \\
            \beta_{L0i} + \beta_{L1i} \times |O|_{ij} + \beta_{L2i} \times PE_{ij} + \beta_{L3i} \times PE_{ij} \times |O|_{ij} + \beta_{\phi i} \times y_{i(j-1)} \mbox{ when } O_{ij} \leq 0
        \end{cases}$$

$$
\boldsymbol{\beta} = (\beta_{w0i}, \beta_{L0i}, \beta_{w1i}, \beta_{L1i}, \beta_{w2i}, \beta_{L2i},  \beta_{w3i}, \beta_{L3i}, \beta_{\phi i}) \sim \mathcal{N}(\boldsymbol{B}, \boldsymbol{\Sigma})
$$

$$
\boldsymbol{B} = 
\begin{bmatrix}
B_{w0} \\
B_{L0} \\
\vdots \\
B_{L3} \\
B_{\phi}
\end{bmatrix}
$$

$$
\boldsymbol{\Sigma} = 
\begin{bmatrix}
\sigma_{1}^2 & \sigma_{12} & \cdots & \sigma_{1n} \\
\sigma_{21} & \sigma_{2}^2 & \cdots & \sigma_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
\sigma_{n1} & \sigma_{n2} & \cdots & \sigma_{n}^2 \\
\end{bmatrix}
$$

## Set the Number of Participants and Observations

In [ ]:
N = 250 # Vector with the number of participants for which compute power
T.obs = 192# Number of trials for each individual
alpha = 0.05 # Significant level
side.test = 3
Opt.Method = 'REML' # Set the optimization method for lme 
R = 50 # Number of Monte Carlo replicates


## Set Predictors

In [ ]:
#set predictors
outcome.true = c(rep(-2,2*24), rep(-1,2*24), rep(1,2*24),rep(2,2*24))
outcome.win = c(rep(c(rep(1,12),rep(2,12)),4),rep(1,2*24),rep(2,2*24))
outcome.loss = c(rep(-2,2*24), rep(-1,2*24),rep(c(rep(-1,12),rep(-2,12)),4))
prob.win = rep(c(rep(0,24),rep(1,24)),4)
prob.loss = rep(c(rep(1,24),rep(0,24)),4)
expectedness = c()
outcome.absolute = abs(outcome.true)
#if negative outcome, then the pe is high when prob.win is high
#vice versa
expectedness <- ifelse(outcome.true < 0, prob.win, prob.loss)

expectedness = rep(expectedness, times = N)
outcome.absolute = rep(outcome.absolute, times = N)
outcome.true = rep(outcome.true, times = N)
outcome.dummy.win = ifelse(outcome.true >0, 1, 0)
outcome.dummy.loss = ifelse(outcome.true >0, 0, 1)

condition = cbind(outcome.absolute, outcome.true, outcome.dummy.win, outcome.dummy.loss, outcome.win, outcome.loss, prob.win, prob.loss, expectedness)
random_indices = sample(nrow(condition))
condition = data.frame(condition[random_indices, ])

condition$pe.win = ifelse(condition$outcome.true<0, 0, condition$expectedness)
condition$pe.loss = ifelse(condition$outcome.true>0, 0, condition$expectedness)
condition$o.abs.win = ifelse(condition$outcome.true<0, 0, condition$outcome.absolute)
condition$o.abs.loss = ifelse(condition$outcome.true>0, 0, condition$outcome.absolute)


## Input Information from the Pilot Study

In [ ]:
# Fixed effects
b00w = 34.00 # Fixed intercept
b00l = 36.34 # Fixed intercept
b01w = 6.84 # Effect of the Level 1 GR winning condition
b10w = 2.47 # Fixed slope Level 1 PE winning condition
b11w = -0.62 # Fixed slope Level 1 GR*PE winning condition
b01l = -6.05 # Effect of the Level 1 GR losing condition
b10l = -7.42 # Fixed slope Level 1 PE losing condition
b11l = 3.17 # Fixed slope Level 1 GR*PE losing condition
b20 = 0.059 # Fixed slop Level 1 AR

# Distribution Level 1 errors
sigma = 11.11# Std. deviation of the Level 1 error
rho = 0 # Autocorrelation of the Level 1 error

# Distribution random effects
sigma.v0w = 11.64# Std. deviation of the random intercept
sigma.v0l = 14.26
sigma.v1w = 2.43 
sigma.v2w = 3.07
sigma.v3w = 3.13
sigma.v1l = 3.26 
sigma.v2l = 5.60
sigma.v3l = 2.48
sigma.v4 = 0.288 

#correlation between every two parameters
rho.v0wv0l =  -0.48
rho.v0wv1w =  -0.23
rho.v0wv2w = -0.25
rho.v0wv3w = 0.01 
rho.v0wv1l = 0.52
rho.v0wv2l = 0.51 
rho.v0wv3l = 0.002
rho.v0wv4 = -0.014

rho.v0lv1w =  -0.63 
rho.v0lv2w = 0.62
rho.v0lv3w = -0.60
rho.v0lv1l = 0.31
rho.v0lv2l = -0.46 
rho.v0lv3l = 0.06
rho.v0lv4 = -0.09

rho.v1wv2w = -0.42
rho.v1wv3w = 0.56
rho.v1wv1l = -0.70 
rho.v1wv2l = 0.08
rho.v1wv3l = -0.06 
rho.v1wv4 = 0.10 

rho.v2wv3w = -0.34
rho.v2wv1l = 0.11 
rho.v2wv2l = -0.68 
rho.v2wv3l =  -0.31 
rho.v2wv4 = -0.045

rho.v3wv1l = -0.60 
rho.v3wv2l = -0.12
rho.v3wv3l = -0.32 
rho.v3wv4 = 0.09

rho.v1lv2l = 0.38
rho.v1lv3l = 0.14
rho.v1lv4 = -0.0947

rho.v2lv3l = 0.33
rho.v2lv4 = -0.008

rho.v3lv4 = -0.04


## Data Simulation

In [ ]:
Sim.Data.ML.5.VAR = function(N,T.obs,
                             b00w,b00l,
                             b01w,b10w,b11w,
                             b01l,b10l,b11l,
                             b20,
                             sigma,
                             sigma.v0w, sigmav0l,
                             sigma.v1w,sigma.v2w,sigma.v3w,
                             sigma.v1l,sigma.v2l,sigma.v3l,
                             sigma.v4,
                             rho.v0wv0l,rho.v0wv1w,rho.v0wv2w,rho.v0wv3w,rho.v0wv1l,rho.v0wv2l,rho.v0wv3l,rho.v0wv4,
                             rho.v0lv1w,rho.v0lv2w,rho.v0lv3w,rho.v0lv1l,rho.v0lv2l,rho.v0lv3l,rho.v0lwv4,
                             rho.v1wv2w,rho.v1wv3w,rho.v1wv1l,rho.v1wv2l,rho.v1wv3l,rho.v1wv4,
                             rho.v2wv3w,rho.v2wv1l,rho.v2wv2l,rho.v2wv3l,rho.v2wv4,
                             rho.v3wv1l,rho.v3wv2l,rho.v3wv3l,rho.v3wv4,
                             rho.v1lv2l,rho.v1lv3l,rho.v1lv4,
                             rho.v2lv3l,rho.v2lv4,    
                             rho.v3lv4){
# Total number of subjects
N = N

# Create variables for the observations
data.IL = expand.grid(Time=1:T.obs)

# Create variable subjno
subjno = expand.grid(1:T.obs,1:N)[,2]
data.IL = cbind(subjno,data.IL)

# Simulate error level-1 (population-level error)
E = rnorm(T.obs*N,0,sigma)

# Simulate error level-2
# Simulate between-subject random effect
# build the variance-covariance matrix
Sigma.v = diag(c(sigma.v0w^2,sigma.v0l^2,sigma.v1w^2,sigma.v2w^2,sigma.v3w^2,sigma.v1l^2,sigma.v2l^2,sigma.v3l^2,sigma.v4^2))
Sigma.v[lower.tri(Sigma.v)] = c(rho.v0wv0l*sigma.v0w*sigma.v0l,
                                rho.v0wv1w*sigma.v0w*sigma.v1w,
                                rho.v0wv2w*sigma.v0w*sigma.v2w,
                                rho.v0wv3w*sigma.v0w*sigma.v3w,
                                rho.v0wv1l*sigma.v0w*sigma.v1l,
                                rho.v0wv2l*sigma.v0w*sigma.v2l,
                                rho.v0wv3l*sigma.v0w*sigma.v3l,
                                rho.v0wv4*sigma.v0w*sigma.v4,
                                rho.v0lv1w*sigma.v0w*sigma.v1w,
                                rho.v0lv2w*sigma.v0l*sigma.v2w,
                                rho.v0lv3w*sigma.v0l*sigma.v3w,
                                rho.v0lv1l*sigma.v0l*sigma.v1l,
                                rho.v0lv2l*sigma.v0l*sigma.v2l,
                                rho.v0lv3l*sigma.v0l*sigma.v3l,
                                rho.v0lv4*sigma.v0l*sigma.v4,
                                rho.v1wv2w*sigma.v1w*sigma.v2w,
                                rho.v1wv3w*sigma.v1w*sigma.v3w,
                                rho.v1wv1l*sigma.v1w*sigma.v1l,
                                rho.v1wv2l*sigma.v1w*sigma.v2l,
                                rho.v1wv3l*sigma.v1w*sigma.v3l,
                                rho.v1wv4*sigma.v1w*sigma.v4,
                                rho.v2wv3w*sigma.v2w*sigma.v3w,
                                rho.v2wv1l*sigma.v2w*sigma.v1l,
                                rho.v2wv2l*sigma.v2w*sigma.v2l,
                                rho.v2wv3l*sigma.v2w*sigma.v3l,
                                rho.v2wv4*sigma.v2w*sigma.v4,
                                rho.v3wv1l*sigma.v3w*sigma.v1l,
                                rho.v3wv2l*sigma.v3w*sigma.v2l,
                                rho.v3wv3l*sigma.v3w*sigma.v3l,
                                rho.v3wv4*sigma.v3w*sigma.v4,
                                rho.v1lv2l*sigma.v1l*sigma.v2l,
                                rho.v1lv3l*sigma.v1l*sigma.v3l,
                                rho.v1lv4*sigma.v1l*sigma.v4,
                                rho.v2lv3l*sigma.v2l*sigma.v3l,
                                rho.v2lv4*sigma.v2l*sigma.v4,
                                rho.v3lv4*sigma.v3l*sigma.v4)

Sigma.v[upper.tri(Sigma.v)] = c(rho.v0wv0l*sigma.v0w*sigma.v0l,
                                rho.v0wv1w*sigma.v0w*sigma.v1w,
                                rho.v0lv1w*sigma.v0l*sigma.v1w,
                                rho.v0wv2w*sigma.v0w*sigma.v2w,
                                rho.v0lv2w*sigma.v0l*sigma.v2w,
                                rho.v1wv2w*sigma.v1w*sigma.v2w,
                                rho.v0wv3w*sigma.v0w*sigma.v3w,
                                rho.v0lv3w*sigma.v0l*sigma.v3w,
                                rho.v1wv3w*sigma.v1w*sigma.v3w,
                                rho.v2wv3w*sigma.v2w*sigma.v3w,
                                rho.v0wv1l*sigma.v0w*sigma.v1l,
                                rho.v0lv1l*sigma.v0l*sigma.v1l,
                                rho.v1wv1l*sigma.v1w*sigma.v1l,
                                rho.v2wv1l*sigma.v2w*sigma.v1l,
                                rho.v3wv1l*sigma.v3w*sigma.v1l,
                                rho.v0wv2l*sigma.v0w*sigma.v2l,
                                rho.v0lv2l*sigma.v0l*sigma.v2l,
                                rho.v1wv2l*sigma.v1w*sigma.v2l,
                                rho.v2wv2l*sigma.v2w*sigma.v2l,
                                rho.v3wv2l*sigma.v3w*sigma.v2l,
                                rho.v1lv2l*sigma.v1l*sigma.v2l,
                                rho.v0wv3l*sigma.v0w*sigma.v3l,
                                rho.v0lv3l*sigma.v0l*sigma.v3l,
                                rho.v1wv3l*sigma.v1w*sigma.v3l,
                                rho.v2wv3l*sigma.v2w*sigma.v3l,
                                rho.v3wv3l*sigma.v3w*sigma.v3l,
                                rho.v1lv3l*sigma.v1l*sigma.v3l,
                                rho.v2lv3l*sigma.v2l*sigma.v3l,
                                rho.v0wv4*sigma.v0w*sigma.v4,
                                rho.v0lv4*sigma.v0l*sigma.v4,
                                rho.v1wv4*sigma.v1w*sigma.v4,
                                rho.v2wv4*sigma.v2w*sigma.v4,
                                rho.v3wv4*sigma.v3w*sigma.v4,
                                rho.v1lv4*sigma.v1l*sigma.v4,
                                rho.v2lv4*sigma.v2l*sigma.v4,
                                rho.v3lv4*sigma.v3l*sigma.v4)

if (eigen(Sigma.v)$values[2] <= 0) {stop('The covariance matrix of the level-2 errors must be positive definite')}

# simulate the participant-wise coefficents
V.i = mvrnorm(N,rep(0,ncol(Sigma.v)),Sigma.v)

V = matrix(0,T.obs*N,ncol(V.i))

#true parameters
for (i in 1:N){
  V[which(data.IL$subjno==i),] = matrix(rep(V.i[i,], each = T.obs), ncol = T.obs)
}

# Set parameters and variables
B00w = rep(b00w,nrow(data.IL))
B00l = rep(b00l,nrow(data.IL))
B01w = rep(b01w,nrow(data.IL))
B10w = rep(b10w,nrow(data.IL))
B11w = rep(b11w,nrow(data.IL))
B01l = rep(b01l,nrow(data.IL))
B10l = rep(b10l,nrow(data.IL))
B11l = rep(b11l,nrow(data.IL))
B20 = rep(b20,nrow(data.IL))


Y = rep(NA, length(subjno))
Y_previous = rep(NA, length(subjno))
Y_previous[sapply(1:N, function(x) which(subjno == x)[1])] = 0


for (i in 1:length(subjno)){
  Y[i] = B00w[i] + B00l[i]*condition$outcome.dummy.win[i] + B01w[i]*condition$o.abs.win[i] + B01l[i]*condition$o.abs.loss[i] + 
    B10w[i]*condition$pe.win[i] + B10l[i]*condition$pe.loss[i] + 
    B11w[i]*(condition$o.abs.win[i]-mean(condition$o.abs.win))*(condition$pe.win[i]-mean(condition$pe.win)) + B11l[i]*(condition$o.abs.loss[i]-mean(condition$o.abs.loss))*(condition$pe.loss[i]-mean(condition$pe.loss)) +
    B20[i]*Y_previous[i] + 
    V[i,1] +  V[i,2] *condition$outcome.dummy.win[i] + V[i,3]*condition$o.abs.win[i] + V[i,6]*condition$o.abs.loss[i] + V[i,4]*condition$pe.win[i] + V[i,7]*condition$pe.loss[i] +
    V[i,5]*(condition$o.abs.win[i]-mean(condition$o.abs.win))*(condition$pe.win[i]-mean(condition$pe.win))+ V[i,8]*(condition$o.abs.loss[i]-mean(condition$o.abs.loss))*(condition$pe.loss[i]-mean(condition$pe.loss)) + V[i,9]*Y_previous[i] + E[i]
  if (is.na(Y_previous[i+1]) & i<length(subjno)) {
    Y_previous[i+1] = Y[i]
  }else{}
}

data = data.frame(cbind(data.IL,Y,condition, Y_previous))

}  


## Fit the Simulated Data Back

In [ ]:
## This function uses Monte Carlo simulations for computing standard errors and statistical power. 

Power.Simulation.ML.5.VAR = function(data,T.obs,
                                     b00w,b00l,
                             b01w,b10w,b11w,
                             b01l,b10l,b11l,
                             b20,
                             sigma,
                             sigma.v0w, sigmav0l,
                             sigma.v1w,sigma.v2w,sigma.v3w,
                             sigma.v1l,sigma.v2l,sigma.v3l,
                             sigma.v4,
                             rho.v0wv0l,rho.v0wv1w,rho.v0wv2w,rho.v0wv3w,rho.v0wv1l,rho.v0wv2l,rho.v0wv3l,rho.v0wv4,
                             rho.v0lv1w,rho.v0lv2w,rho.v0lv3w,rho.v0lv1l,rho.v0lv2l,rho.v0lv3l,rho.v0lwv4,
                             rho.v1wv2w,rho.v1wv3w,rho.v1wv1l,rho.v1wv2l,rho.v1wv3l,rho.v1wv4,
                             rho.v2wv3w,rho.v2wv1l,rho.v2wv2l,rho.v2wv3l,rho.v2wv4,
                             rho.v3wv1l,rho.v3wv2l,rho.v3wv3l,rho.v3wv4,
                             rho.v1lv2l,rho.v1lv3l,rho.v1lv4,
                             rho.v2lv3l,rho.v2lv4,    
                             rho.v3lv4,
                                     alpha,
                                     side.test,
                                     Opt.Method){
  # Fit linear mixed-effects models
  
  if (Opt.Method == 'ML'){
    # Maximum Likelihood
    fit.lme = try(lme(Y ~ outcome.dummy.win + o.abs.win + o.abs.loss + pe.win + pe.loss + o.abs.win * pe.win + o.abs.loss * pe.loss + Y_previous, 
                  random = ~ outcome.dummy.win  + o.abs.win + o.abs.loss + pe.win + pe.loss + o.abs.win * pe.win + o.abs.loss * pe.loss + Y_previous|subjno,correlation = corAR1(),data=data,na.action=na.omit,method='REML',
                  control=lmeControl(opt='optim')),silent = FALSE)

  }
  
  if (Opt.Method == 'REML'){
    fit.lme = try(lme(Y ~ outcome.dummy.win + o.abs.win + o.abs.loss + pe.win + pe.loss + o.abs.win * pe.win + o.abs.loss * pe.loss + Y_previous, 
                  random = ~ outcome.dummy.win  + o.abs.win + o.abs.loss + pe.win + pe.loss + o.abs.win * pe.win + o.abs.loss * pe.loss + Y_previous|subjno,correlation = corAR1(),data=data,na.action=na.omit,method='REML',
                  control=lmeControl(opt='optim')),silent = FALSE)

  }
  
  if (length(fit.lme)>1){
    
    # Obtain the estimated coefficients of the model
    beta.hat.lme = coef(summary(fit.lme))[,'Value']
    
    # Obtain the standard errors
    StdError.beta.lme = coef(summary(fit.lme))[,'Std.Error']
    
    # Compute power and standard error from lme  
    
    if (side.test == 1){ # One-side test: positive
      p.value = pt(coef(summary(fit.lme))[,4], coef(summary(fit.lme))[,3], lower = FALSE)
    }
    
    if (side.test == 2){ # One-side test: negative
      p.value = pt(coef(summary(fit.lme))[,4], coef(summary(fit.lme))[,3], lower = TRUE)
      
    }
    
    if (side.test == 3){ # Two-tailed test
      p.value = 2*pt(-abs(coef(summary(fit.lme))[,4]), coef(summary(fit.lme))[,3])
    }
    
    power.hat.lme = p.value < alpha
    
    return(list(beta.hat.lme=beta.hat.lme,
                power.hat.lme=power.hat.lme,
                StdError.beta.lme=StdError.beta.lme,
                p.value=p.value))}
  
  if (length(fit.lme)==1){
    return(list(fit.lme))
  }}

#############################
######Power Analysis#########
Power.Simulation.Estimates.ML.5.VAR = function(N,T.obs,
                                               b00w,b00l,
                             b01w,b10w,b11w,
                             b01l,b10l,b11l,
                             b20,
                             sigma,
                             sigma.v0w, sigmav0l,
                             sigma.v1w,sigma.v2w,sigma.v3w,
                             sigma.v1l,sigma.v2l,sigma.v3l,
                             sigma.v4,
                             rho.v0wv0l,rho.v0wv1w,rho.v0wv2w,rho.v0wv3w,rho.v0wv1l,rho.v0wv2l,rho.v0wv3l,rho.v0wv4,
                             rho.v0lv1w,rho.v0lv2w,rho.v0lv3w,rho.v0lv1l,rho.v0lv2l,rho.v0lv3l,rho.v0lwv4,
                             rho.v1wv2w,rho.v1wv3w,rho.v1wv1l,rho.v1wv2l,rho.v1wv3l,rho.v1wv4,
                             rho.v2wv3w,rho.v2wv1l,rho.v2wv2l,rho.v2wv3l,rho.v2wv4,
                             rho.v3wv1l,rho.v3wv2l,rho.v3wv3l,rho.v3wv4,
                             rho.v1lv2l,rho.v1lv3l,rho.v1lv4,
                             rho.v2lv3l,rho.v2lv4,    
                             rho.v3lv4,
                                               alpha,
                                               side.test,
                                               Opt.Method,R){
  
  tic()
  # Simulate data from the linear mixed-effects model  
  data = lapply(1:R, function(r) 
    Sim.Data.ML.5.VAR(N,T.obs,
                      b00w,b00l,
                             b01w,b10w,b11w,
                             b01l,b10l,b11l,
                             b20,
                             sigma,
                             sigma.v0w, sigmav0l,
                             sigma.v1w,sigma.v2w,sigma.v3w,
                             sigma.v1l,sigma.v2l,sigma.v3l,
                             sigma.v4,
                             rho.v0wv0l,rho.v0wv1w,rho.v0wv2w,rho.v0wv3w,rho.v0wv1l,rho.v0wv2l,rho.v0wv3l,rho.v0wv4,
                             rho.v0lv1w,rho.v0lv2w,rho.v0lv3w,rho.v0lv1l,rho.v0lv2l,rho.v0lv3l,rho.v0lwv4,
                             rho.v1wv2w,rho.v1wv3w,rho.v1wv1l,rho.v1wv2l,rho.v1wv3l,rho.v1wv4,
                             rho.v2wv3w,rho.v2wv1l,rho.v2wv2l,rho.v2wv3l,rho.v2wv4,
                             rho.v3wv1l,rho.v3wv2l,rho.v3wv3l,rho.v3wv4,
                             rho.v1lv2l,rho.v1lv3l,rho.v1lv4,
                             rho.v2lv3l,rho.v2lv4,    
                             rho.v3lv4)) 
  
  # Simulation-based power analysis using Monte Carlo simulation
  fit.list.sim = lapply(1:R, function(r) Power.Simulation.ML.5.VAR(data[[r]],T.obs,
                             b00w,b00l,
                             b01w,b10w,b11w,
                             b01l,b10l,b11l,
                             b20,
                             sigma,
                             sigma.v0w, sigmav0l,
                             sigma.v1w,sigma.v2w,sigma.v3w,
                             sigma.v1l,sigma.v2l,sigma.v3l,
                             sigma.v4,
                             rho.v0wv0l,rho.v0wv1w,rho.v0wv2w,rho.v0wv3w,rho.v0wv1l,rho.v0wv2l,rho.v0wv3l,rho.v0wv4,
                             rho.v0lv1w,rho.v0lv2w,rho.v0lv3w,rho.v0lv1l,rho.v0lv2l,rho.v0lv3l,rho.v0lwv4,
                             rho.v1wv2w,rho.v1wv3w,rho.v1wv1l,rho.v1wv2l,rho.v1wv3l,rho.v1wv4,
                             rho.v2wv3w,rho.v2wv1l,rho.v2wv2l,rho.v2wv3l,rho.v2wv4,
                             rho.v3wv1l,rho.v3wv2l,rho.v3wv3l,rho.v3wv4,
                             rho.v1lv2l,rho.v1lv3l,rho.v1lv4,
                             rho.v2lv3l,rho.v2lv4,    
                             rho.v3lv4,
                             alpha,
                             side.test,
                             Opt.Method))
  toc(log = TRUE, quiet = TRUE)
  log.txt =  tic.log(format = TRUE)
  log.lst = tic.log(format = FALSE)
  tic.clearlog()
  timings.sim.power.simulation = unlist(lapply(log.lst, function(x) x$toc - x$tic))
  
  # Get a vector with the iterations that converge
  errors = rep(0,R)
  for (r in 1:R){errors[r] = length(fit.list.sim[[r]])}
  
  R.converge = which(errors>1)
  
  # Number of replicates that converge
  n.R = length(R.converge)
  
  # Estimates the fixed effects
  beta.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) fit.list.sim[[r]]$beta.hat.lme)), 
                             ncol=9, byrow=TRUE)
  colnames(beta.hat.lme.list) =  c('Intercept.loss','Intercept.win','GR.win','GR.loss','PE.win','PE.loss','GR*PE.win','GR*PE.loss','AR')
  
  beta.hat.lme = colMeans(beta.hat.lme.list)
  beta.hat.lme.se = apply(beta.hat.lme.list,2,sd)/sqrt(n.R)
  
  # Power
  power.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) 
    fit.list.sim[[r]]$power.hat.lme)), 
    ncol=9, byrow=TRUE)
  colnames(power.hat.lme.list) =   c('Intercept.loss','Intercept.win','GR.win','GR.loss','PE.win','PE.loss','AR','GR*PE.win','GR*PE.loss')
  
  power.hat.lme = colMeans(power.hat.lme.list)
  power.hat.lme.se = sqrt(power.hat.lme*(1-power.hat.lme))/sqrt(n.R)
  
  # Standard errors
  StdError.beta.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) 
    fit.list.sim[[r]]$StdError.beta.lme)), 
    ncol=9, byrow=TRUE)
  colnames(StdError.beta.hat.lme.list) =  c('Intercept.loss','Intercept.win','GR.win','GR.loss','PE.win','PE.loss','AR','GR*PE.win','GR*PE.loss')
  
  StdError.beta.hat.lme = colMeans(StdError.beta.hat.lme.list)
  StdError.beta.hat.lme.se = apply(StdError.beta.hat.lme.list,2,sd)/sqrt(n.R)
  
  # P-value
  p.value.beta.hat.lme.list = matrix(unlist(lapply(R.converge, function(r) 
    fit.list.sim[[r]]$p.value)), 
    ncol=9, byrow=TRUE)
  colnames(StdError.beta.hat.lme.list) =   c('Intercept.loss','Intercept.win','GR.win','GR.loss','PE.win','PE.loss','AR','GR*PE.win','GR*PE.loss')
  
  return(list(beta.hat.lme.list=beta.hat.lme.list,
              beta.hat.lme=beta.hat.lme,
              beta.hat.lme.se=beta.hat.lme.se,
              power.hat.lme.list=power.hat.lme.list,
              power.hat.lme=power.hat.lme,
              power.hat.lme.se=power.hat.lme.se,
              StdError.beta.hat.lme=StdError.beta.hat.lme,
              StdError.beta.hat.lme.se=StdError.beta.hat.lme.se,
              p.value.beta.hat.lme.list=p.value.beta.hat.lme.list,
              timings.sim.power.simulation=timings.sim.power.simulation,
              n.R=n.R))}


## Power Estimation

In [ ]:
#set.seed(1) # Set seed or the random number generator
# Function for computing power using simulation-based approach using REML
sim.power.list = Power.Simulation.Estimates.ML.5.VAR(N,T.obs,
                             b00w,b00l,
                             b01w,b10w,b11w,
                             b01l,b10l,b11l,
                             b20,
                             sigma,
                             sigma.v0w, sigmav0l,
                             sigma.v1w,sigma.v2w,sigma.v3w,
                             sigma.v1l,sigma.v2l,sigma.v3l,
                             sigma.v4,
                             rho.v0wv0l,rho.v0wv1w,rho.v0wv2w,rho.v0wv3w,rho.v0wv1l,rho.v0wv2l,rho.v0wv3l,rho.v0wv4,
                             rho.v0lv1w,rho.v0lv2w,rho.v0lv3w,rho.v0lv1l,rho.v0lv2l,rho.v0lv3l,rho.v0lwv4,
                             rho.v1wv2w,rho.v1wv3w,rho.v1wv1l,rho.v1wv2l,rho.v1wv3l,rho.v1wv4,
                             rho.v2wv3w,rho.v2wv1l,rho.v2wv2l,rho.v2wv3l,rho.v2wv4,
                             rho.v3wv1l,rho.v3wv2l,rho.v3wv3l,rho.v3wv4,
                             rho.v1lv2l,rho.v1lv3l,rho.v1lv4,
                             rho.v2lv3l,rho.v2lv4,    
                             rho.v3lv4,
                             alpha,
                             side.test,
                             Opt.Method="REML",
                             R)


# Output function
sim.power.list

# Estimated statistical power
sim.power.list$power.hat.lme 